<a href="https://colab.research.google.com/github/raw-fun/AI/blob/main/Copy_of_web_scrap_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ১. প্রো-লেভেল সেটআপ
!pip install -q playwright pandas
!playwright install chromium
!playwright install-deps chromium

import asyncio
import re
import random
import pandas as pd
from playwright.async_api import async_playwright
from urllib.parse import urljoin, urlparse
import datetime

print("🚀 অ্যাডভান্সড ফরেনসিক ইঞ্জিন প্রস্তুত।")

In [ ]:
import asyncio
import re
import random
from urllib.parse import urljoin, urlparse

class DeepLinkForensicFinal:
    def __init__(self, root_url, max_pages=50):
        self.root_url = root_url.rstrip('/')
        self.max_pages = max_pages
        self.nodes = [] #
        self.visited = set()
        self.queue = [root_url]

    def is_valid_page(self, url):
        """শুধুমাত্র আসল পেজগুলো স্ক্যান করবে, ফাইল নয়"""
        parsed = urlparse(url)
        # বাদ দেওয়ার জন্য ফাইল এক্সটেনশন লিস্ট
        excluded_ext = ('.css', '.js', '.ico', '.png', '.jpg', '.jpeg', '.svg', '.gif', '.pdf', '.woff', '.woff2')
        # যদি ইউআরএলটি এই এক্সটেনশনগুলোর কোনোটি দিয়ে শেষ হয়, তবে সেটি বাতিল
        if parsed.path.lower().endswith(excluded_ext):
            return False
        # শুধুমাত্র সরকারি ডোমেইন চেক
        return ".gov.bd" in parsed.netloc

    async def extract_links_clean(self, page, current_url):
        """অপ্রয়োজনীয় ফাইল বাদ দিয়ে শুধুমাত্র পেজ লিঙ্ক সংগ্রহ করবে"""
        try:
            content = await page.content()
            # রেজেক্স ব্যবহার করে সব লিঙ্ক বের করা
            links = re.findall(r'href=[\'"]?([^\'" >]+)', content)

            added = 0
            for link in set(links):
                full_url = urljoin(current_url, link).split('#')[0].split('?')[0].rstrip('/')

                if full_url not in self.visited and self.is_valid_page(full_url):
                    if len(self.visited) + len(self.queue) < self.max_pages + 50:
                        self.queue.append(full_url)
                        added += 1
            return added
        except:
            return 0

    async def run(self):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True, args=["--disable-blink-features=AutomationControlled"])
            context = await browser.new_context(
                user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
            )

            print(f"🕵️ মিশন স্ট্যাটাস: {self.root_url} এ ক্লিন স্ক্যান শুরু...")
            print("-" * 75)

            while self.queue and len(self.nodes) < self.max_pages:
                url = self.queue.pop(0)
                if url in self.visited: continue
                self.visited.add(url)

                page = await context.new_page()
                # রিসোর্স ব্লকিং
                await page.route("**/*.{css,js,png,jpg,ico,woff,svg}", lambda r: r.abort())

                success = False
                retries = 2

                while retries > 0 and not success:
                    try:
                        print(f"🔍 [{len(self.nodes)+1}] ভিজিট: {url[:60]}...", end=" ")

                        # নেভিগেশন সেটিংস
                        response = await page.goto(url, wait_until="domcontentloaded", timeout=45000)
                        await asyncio.sleep(2) # রেন্ডারিং বিরতি

                        status = response.status if response else 200

                        if status == 521 or status == 504:
                            print(f"⏳ [Status {status}] পুনরায় চেষ্টা করা হচ্ছে...", end=" ")
                            retries -= 1
                            await asyncio.sleep(5)
                            continue

                        self.nodes.append({"url": url, "status": status, "domain": urlparse(url).netloc})
                        print(f"✅ [{status}]")

                        # লিঙ্ক সংগ্রহ
                        new_nodes = await self.extract_links_clean(page, url)
                        if new_nodes > 0: print(f"   ∟ 🧬 {new_nodes}টি ভ্যালিড নোড কিউতে যুক্ত।")

                        success = True
                    except Exception:
                        retries -= 1
                        if retries == 0:
                            print("❌ স্কিপড (টাইমআউট)")
                            self.nodes.append({"url": url, "status": "Timed Out", "domain": urlparse(url).netloc})

                await page.close()
                await asyncio.sleep(random.uniform(1, 2)) # হিউম্যান ডিল্

            await browser.close()
            return self.nodes

In [ ]:
# মিশন প্যারামিটার
TARGET = "https://bangladesh.gov.bd" #@param {type:"string"}
TOTAL_PAGES = 1000 #@param {type:"integer"}

async def start_clean_mission():
    engine = DeepLinkForensicFinal(TARGET, max_pages=TOTAL_PAGES)
    results = await engine.run()

    df = pd.DataFrame(results)
    if df.empty: return

    print("\n" + "█"*75)
    print("              🏁 FINAL CLEAN FORENSIC REPORT")
    print("█"*75)
    print(f"🔹 মোট প্রসেসড ইউআরএল  : {len(df)}")
    print(f"🔹 সফল কানেকশন (200)   : {len(df[df['status'] == 200])}")
    print(f"🔹 ডোমেইন ডাইভারসিটি    : {df['domain'].nunique()} টি")
    print("-" * 75)

    display(df.head(20))
    df.to_csv("clean_forensic_results.csv", index=False)

await start_clean_mission()